# Role: Hospital-MedicalDevice
### This notebook acts like an issuer in blockchain. What we do is to generate commands and write them to medical devices wallet. 


## 1. Initialise a controller for Hospital Agent

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8072
ADMIN_URL = "http://hospital-agent:8071"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)




IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

In [3]:
response = await agent_controller.wallet.get_public_did()
print(response)

#Todo: check the ownership of medical device


#put the schema_id of medical device when you finish the process of transfering the ownership
#note that the schema id must contain the attributes of operation commands
issuer_nym = response['result']['did']
print('\nIssuer public DID:',issuer_nym)

issuer_verkey = await agent_controller.ledger.get_did_verkey(issuer_nym)
print(issuer_verkey)

issuer_endpoint = await agent_controller.ledger.get_did_endpoint(issuer_nym)
print(issuer_endpoint)


{'result': None}


TypeError: 'NoneType' object is not subscriptable

## 9. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


## 3a. Use a previously written schema

In [4]:
# schema_id = 'EuEtnVakYFyBtGFT1nHYtH:2:SSI PyDentity Tutorial:0.0.1'
schema_id = issuer_nym

NameError: name 'issuer_nym' is not defined

## 3b. (OPTIONAL) Write a Schema to the Ledger

For more details see the [schema-api notebook](http://localhost:8888/lab/tree/schema_api.ipynb)

**Note: You will only be able to do this once unless you change the schema_name or version. Once you have a schema written to the ledger it can be reused by multiple entities**

In [ ]:
# # Define you schema name - must be unique on the ledger
# schema_name = "SSI PyDentity Tutorial"
# # Can version the schema if you wish to update it
# schema_version = "0.0.1"
# # Define any list of attributes you wish to include in your schema
# attributes = ["fullname", "skill", "age"]

# response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
# schema_id = response["schema_id"]
# print(schema_id)


## 4. Write a Credential Definition to the Ledger

**Note: Again this can only be done once per issuer, per schema_id.**

In [8]:
# response = await agent_controller.definitions.write_cred_def(schema_id)

# cred_def_id = response["credential_definition_id"]
# print(cred_def_id)

cred_def_id = 'XXX' #Please put the credential definition id you register in manufacturer notebook
print(cred_def_id)


NameError: name 'schema_id' is not defined

## 12. Populate the Attribues to Issue to Medical Device Controller Holder (Hospital)

We will issue a credential to the identity holder consisting of the following attributes:

credential_attributes = [
    {"name": "fullname", "value": name},
    {"name": "skill", "value": "PyDentity SSI Ninja"},
    {"name": "age", "value": age}
]

The notebook will ask you to input the identity device's full name and age which will be used to populate the schema above with the identity holders attribute information.

In [ ]:
from datetime import datetime

ProductionDate=input("Please enter production date: ")
ExpirationDate=input("Please enter expiration date: ")
# datetime_str = '09/19/22'

Manufacturer=input("Please enter manufacturer: ")
Owner=input("Please enter owner: ")
Type=input("Please enter type: ")
Status=input("Please enter status: ")

credential_attributes = [
    {"name": "ProductionDate", "value": datetime.strptime(ProductionDate, '%m/%d/%y')},
    {"name": "ExpirationDate", "value": datetime.strptime(ExpirationDate, '%m/%d/%y')},
    {"name": "Manufacturer", "value": Manufacturer},
    {"name": "Owner", "value": Owner},
    {"name": "Type", "value": Type},
    {"name": "Status", "value": Status},
]
print(credential_attributes)

## 8a. Check for any existing connections

In [1]:
# Check for existing connections
connection = await agent_controller.connections.get_connections()
print("EXISTING CONNECTIONS")
for key, value in connection.items():
    for item in value:
        print('ConnectionID:', item['connection_id'], 'Status:',item['state'])

NameError: name 'agent_controller' is not defined

## 8b. Create an Invitation

In [ ]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation")
print(invite['invitation_url'])
inviteURL = invite['invitation_url']

## 8c. Generate QR Code to be scanned with Mobile SSI Wallet

In [ ]:
import qrcode
# Link for connection invitation
input_data = inviteURL
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=10,
        border=5)
qr.add_data(input_data)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img.save('issuer_agent_invite_QRcode.png')

from IPython.display import Image
Image(width=400, filename='./issuer_agent_invite_QRcode.png')

## 8d. Check if established connection is in active state

In [ ]:
import time

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'active':
#     print('ConnectionId {0} is not in active state yet'.format(connection_id))
    print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
    trust_ping = await agent_controller.messaging.trust_ping(connection_id,'hello!')
#     print('Trust ping send to ConnectionId {0} to activate connection'.format(trust_ping))
    print(colored("Trust ping send to ConnectionId {0} to activate connection".format(trust_ping), "blue", attrs=["bold"]))
    time.sleep(5)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))


## 9. Send Credential

This sends a credential to a identity holder (User), and automates the rest of the protocol.

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [ ]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")
#Please record the credential id and past them to the medical device computer notebook, 
#you will need to fetch them from wallet to device computer to run the corresponding operation



## 10. Accept credential in Mobile SSI Wallet

## 11. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)

# Proceed to controller on [Medical Device Controller Notebook](http://localhost:8891/lab/tree/Medical_Device_Computer.ipynb)
